# Çokgen Algılamak
> ##### gerekli kütüphaneler import edildi

In [8]:
import cv2
import numpy as np

In [37]:
font = cv2.FONT_HERSHEY_SIMPLEX
font1 = cv2.FONT_HERSHEY_COMPLEX

img = cv2.imread("resimler/1.1 polygons.png.png")

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # gri resim elde ettik

ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY) #threshold ile resimi siyah beyaza çevirdik

contours,_ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #toplamda iki değer döndürüyor son değer bizim işimize yaramıyor şuanlık

for i in contours:
    epsilon = 0.01*cv2.arcLength(i,True)
    approx = cv2.approxPolyDP(i,epsilon,True)
    #print("approx",approx)
    cv2.drawContours(img,[approx],-1,(0),2) 
    
    
    print("x: ",x)
    print("y: ",y)
    print(len(approx))
    
    if len(approx) == 3:
        cv2.putText(img,"Ucgen",(x,y),font,1,(0))
    elif len(approx) == 4:
        cv2.putText(img,"Kare",(x,y),font,1,(0))
    elif len(approx) == 5:
        cv2.putText(img,"Besgen",(x,y),font,1,(0))
    elif len(approx) == 6:
        cv2.putText(img,"Altigen",(x,y),font,1,(0))
    else :
        cv2.putText(img,"Elips",(x,y),font,1,(0))
    

cv2.imshow("contour",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

x:  0
y:  0
4
x:  27
y:  318
12
x:  385
y:  245
6
x:  130
y:  69
3
x:  279
y:  68
4
x:  610
y:  53
5


# Kameradan çokgen Algılamak

In [45]:
import requests
import cv2
import numpy as np
#telefonun kamerasına bağlanıyoruz
#url = "http://192.168.137.13:8080//shot.jpg"

cap = cv2.VideoCapture(0) # videocaptura sıfır diyerek kameradan görüntü alacağımızı belirtilir
font = cv2.FONT_HERSHEY_SIMPLEX
font1 = cv2.FONT_HERSHEY_COMPLEX


def nothing(x):
        
        pass
    
cv2.namedWindow("Trackbar") # yeniden adlandırıldı
cv2.resizeWindow ("Trackbar",500,500)

cv2.createTrackbar("lower - H","Trackbar",0,180,nothing) # trackbara tuşlar ve görevler atandı
cv2.createTrackbar("lower - S","Trackbar",0,255,nothing) # adı G ve değişim aralığı 0-255
cv2.createTrackbar("lower - V","Trackbar",0,255,nothing)

cv2.createTrackbar("upper - H","Trackbar",0,180,nothing)
cv2.createTrackbar("upper - S","Trackbar",0,255,nothing)
cv2.createTrackbar("upper - V","Trackbar",0,255,nothing)

cv2.setTrackbarPos("upper - H","Trackbar",180)
cv2.setTrackbarPos("upper - S","Trackbar",255)
cv2.setTrackbarPos("upper - V","Trackbar",255)


# her bir kare sonsuz bir while döngüsü ile yakalayarak video oluşturuyoruz.
while True:
    #img_resp = requests.get(url) # Url'ye istek gönderiyoruz ve url'den görüntüleri alıyoruz
    #img_arr = np.array(bytearray(img_resp.content), dtype = np.uint8) #array olarak dönüştürüyoruz
    #img = cv2.imdecode(img_arr,cv2.IMREAD_COLOR) 
    #img = cv2.resize(img,(640,480))
    ret,frame = cap.read() # cap.read() sağlıklı bi şekilde görüntü okuduğunda true değerleri göndererek ret hafızasında saklanır
    
    #flipcode = 0: flip vertically
    #flipcode > 0: flip horizontally
    #flipcode < 0: flip vertically and horizontally
    
    frame = cv2.flip(frame,1)  #cv2.flip kodu ile görüntünün eksenlere göre tersini alabiliriz
    
    frame_hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    lower_h = cv2.getTrackbarPos("lower - H","Trackbar") #tuşlara atadığımız özellikleri ekrana atıyoruz
    lower_s = cv2.getTrackbarPos("lower - S","Trackbar")
    lower_v = cv2.getTrackbarPos("lower - V","Trackbar")
    
    upper_h = cv2.getTrackbarPos("upper - H","Trackbar")
    upper_s = cv2.getTrackbarPos("upper - S","Trackbar")
    upper_v = cv2.getTrackbarPos("upper - V","Trackbar")
    
    lower_color = np.array([lower_h,lower_s,lower_v])
    upper_color = np.array([upper_h,upper_s,upper_v])
    
    mask = cv2.inRange(frame_hsv,lower_color,upper_color)
    
    kernel = np.ones((5,5),np.uint8)
    kernel = cv2.medianBlur(kernel,3)
    mask = cv2.erode(mask,kernel)
    
    contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE) #toplamda iki değer döndürüyor son değer bizim işimize yaramıyor şuanlık

    for i in contours:
        area = cv2.contourArea(i)
        epsilon = 0.02*cv2.arcLength(i,True)
        approx = cv2.approxPolyDP(i,epsilon,True)
        
        x = approx.ravel()[0] #ravel sütunları Satıra dönüştürür
        y = approx.ravel()[1] #böyle yaparak şekillerin ilk başlangıç konumlarını belirlemiş olacağız ve o konumlara --
        # --isimleri yazdıracağız
        
        if area > 400: #alanı dötyüzün altında olanları çizmesin
            cv2.drawContours(frame,[approx],-1,(0),3) 
            if len(approx) == 3:
                cv2.putText(frame,"Ucgen",(x,y),font,2,(0,255,0))
            elif len(approx) == 4:
                cv2.putText(frame,"Dortgen",(x,y),font,2,(0,255,0))
            elif len(approx) > 6:
                cv2.putText(frame,"Cember",(x,y),font,2,(0,255,0))
    
    cv2.imshow("Original",frame)
    cv2.imshow("Mask",mask)
    
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(1) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break

cap.release() 
cv2.destroyAllWindows()

# Arka Plan çıkarmak
> ### 1. Yol

In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture("resimler/5.1 car.mp4.mp4")

ret,frame_1 = cap.read() #ilk kare
frame_1 = cv2.resize(frame_1,(640,480))
gray_1 = cv2.cvtColor(frame_1,cv2.COLOR_BGR2GRAY)
gray_1 = cv2.GaussianBlur(gray_1,(5,5),0)
while True:
    
    ret,frame = cap.read()
    frame = cv2.resize(frame,(640,480))
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(5,5),0)
    
    diff = cv2.absdiff(gray_1,gray) #iki kare arasındaki farkı bulucak
    _,diff = cv2.threshold(diff,25,255,cv2.THRESH_BINARY)
    
    cv2.imshow("frame",frame)
    cv2.imshow("first",frame_1)
    cv2.imshow("diff",diff)
 
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(20) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break
    
cap.release() 
cv2.destroyAllWindows()

> ### 2.Yol

In [7]:
import cv2
import numpy as np

cap = cv2.VideoCapture("resimler/5.1 car.mp4.mp4")
subtractor = cv2.createBackgroundSubtractorMOG2(history=100,varThreshold=100,detectShadows=True)

while True:
    
    ret,frame = cap.read()
    frame = cv2.resize(frame,(640,480))
    mask = subtractor.apply(frame)
    
    
    
    cv2.imshow("frame",frame)
    cv2.imshow("mask",mask)
    
 
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(10) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break
    
cap.release() 
cv2.destroyAllWindows()

# Mouse Kullanımı

In [1]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)


circles =[]
def mouse(event,x,y,flags,params):
    if event == cv2.EVENT_LBUTTONDOWN: #SOL TUŞA BASILDI İSE
        circles.append((x,y))
        

cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame",mouse)

while True:
    
    ret,frame = cap.read() #kareleri oku
    frame = cv2.resize(frame,(640,480))
    if ret==0:
        break
    for center in circles:
        cv2.circle(frame,center,20,(0,255,0),-1)
    
    cv2.imshow("Frame",frame)
    
    key = cv2.waitKey(1)
 
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if key == 27 : # esc  tuşuna basınca çık
        break
    elif key == ord("q"):
        circles = []
    
cap.release() 
cv2.destroyAllWindows()

# Yüz özelliklerini kullanmak

In [17]:
import cv2
import numpy as np
import math

#roi alanı içresindeki küçük beyaz alanlarla ilgilenmemek için 
#max kontur olan yüzümüzü tepit etmek için fonksiyon tanımlıyoruz

def findMaxContour(contours):
    max_i =0
    max_area=0
    
    for i in range(len(contours)):
        area_face = cv2.contourArea(contours[i]) #contourların alanını buluyor
        if max_area < area_face:
            max_area = area_face
            max_i =i
        try:
            c = contours[max_i]
        except:
            contours =[0]
            c = contours[0]
        return c

cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    frame =cv2.flip(frame,1)
    
    #roi ilgili bölge ile yüz hatlarımızı görüntüden ayırdık
    roi = frame[100:300,250:450] # frame[y1:y2,x1:x2] ilgili alan roi
    cv2.rectangle(frame,(250,100),(450,300),(0,255,0),0)
    
    
    #ten rengin hsv kodu çıkarılmaya çalışıldı
    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    lower_color = np.array([0,99,44],dtype=np.uint8)
    upper_color = np.array([21,255,255],dtype=np.uint8)
    
    mask=cv2.inRange(hsv,lower_color,upper_color)
    kernel = np.ones((3,3),np.uint8) #birlerden oluşup beyazlar içindeki siyahları yok etmek için kullanılacak
    mask = cv2.dilate(mask,kernel,iterations =1)
    mask = cv2.medianBlur(mask,15)
    
    contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
    
        try:
            c = findMaxContour(contours) 
            extLeft = tuple(c[c[:,:,0].argmin()][0]) # en küçük x bulunacak ve konumu belirlenecek
            #print("left: ",extLeft)
            extRight = tuple(c[c[:,:,0].argmax()][0])
            extTop = tuple(c[c[:,:,1].argmin()][0]) # en küçük y bulunucak ve konumu belirlenecek
            extBot = tuple(c[c[:,:,1].argmax()][0])

            cv2.circle(roi,extLeft,5,(0,255,0),2)
            cv2.circle(roi,extRight,5,(0,255,0),2)
            cv2.circle(roi,extTop,5,(0,255,0),2)
            cv2.circle(roi,extBot,5,(0,255,0),2)
            
            cv2.line(roi,extLeft,extTop,(255,0,0),2)
            cv2.line(roi,extTop,extRight,(255,0,0),2)
            cv2.line(roi,extRight,extBot,(255,0,0),2)
            cv2.line(roi,extBot,extLeft,(255,0,0),2)
            
            
            #iki vektör arasındaki açıyı bulmak
            a = math.sqrt((extRight[0]-extTop[0])**2+(extRight[1]-extTop[1])**2)
            b = math.sqrt((extBot[0]-extRight[0])**2+(extBot[1]-extRight[1])**2)
            c = math.sqrt((extBot[0]-extTop[0])**2+(extBot[1]-extTop[1])**2)
            
            try:
                angle_ab = int(math.acos((a**2+b**2-c**2)/(2*b*c))*57)
                cv2.putText(roi,str(angle_ab),((extRight[0]-50),extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
            except:
                cv2.putText(roi,"?",((extRight[0]-50),extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)

        except:
            pass

    else:
        pass
    
    cv2.imshow("frame",frame)
    cv2.imshow("roi",roi)
    cv2.imshow("mask",mask)
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(1) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break
        
cap.release() 
cv2.destroyAllWindows()

In [48]:
import cv2
import numpy as np
import math

#roi alanı içresindeki küçük beyaz alanlarla ilgilenmemek için 
#max kontur olan yüzümüzü tepit etmek için fonksiyon tanımlıyoruz

def findMaxContour(contours):
    max_i =0
    max_area=0
    
    for i in range(len(contours)):
        area_hand = cv2.contourArea(contours[i]) #contourların alanını buluyor
        if max_area < area_hand:
            max_area = area_hand
            max_i =i
        try:
            c = contours[max_i]
        except:
            contours =[0]
            c = contours[0]
        return c

cap = cv2.VideoCapture(0)

while True:
    ret,frame = cap.read()
    frame =cv2.flip(frame,1)
    
    #roi ilgili bölge ile yüz hatlarımızı görüntüden ayırdık
    roi = frame[100:300,250:450] # frame[y1:y2,x1:x2] ilgili alan roi
    cv2.rectangle(frame,(250,100),(450,300),(0,255,0),0)
    
    
    #ten rengin hsv kodu çıkarılmaya çalışıldı
    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)
    lower_color = np.array([0,99,44],dtype=np.uint8)
    upper_color = np.array([21,255,255],dtype=np.uint8)
    
    mask=cv2.inRange(hsv,lower_color,upper_color)
    kernel = np.ones((3,3),np.uint8) #birlerden oluşup beyazlar içindeki siyahları yok etmek için kullanılacak
    mask = cv2.dilate(mask,kernel,iterations =1)
    mask = cv2.medianBlur(mask,15)
    
    contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
    
        try:
            c = findMaxContour(contours) 
            extLeft = tuple(c[c[:,:,0].argmin()][0]) # en küçük x bulunacak ve konumu belirlenecek
            #print("left: ",extLeft)
            extRight = tuple(c[c[:,:,0].argmax()][0])
            extTop = tuple(c[c[:,:,1].argmin()][0]) # en küçük y bulunucak ve konumu belirlenecek
            

            cv2.circle(roi,extLeft,5,(0,255,0),2)
            cv2.circle(roi,extRight,5,(0,255,0),2)
            cv2.circle(roi,extTop,5,(0,255,0),2)
            
            
            cv2.line(roi,extLeft,extTop,(255,0,0),2)
            cv2.line(roi,extTop,extRight,(255,0,0),2)
            cv2.line(roi,extRight,extLeft,(255,0,0),2)
            
            
            
            #iki vektör arasındaki açıyı bulmak
            a = math.sqrt((extRight[0]-extTop[0])**2+(extRight[1]-extTop[1])**2)
            b = math.sqrt((extBot[0]-extRight[0])**2+(extBot[1]-extRight[1])**2)
            c = math.sqrt((extBot[0]-extTop[0])**2+(extBot[1]-extTop[1])**2)
            
            try:
                angle_ab = int(math.acos((a**2+b**2-c**2)/(2*b*c))*57)
                cv2.putText(roi,str(angle_ab),((extRight[0]-50),extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
            except:
                cv2.putText(roi,"?",((extRight[0]-50),extRight[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
                
            if angle_ab > 70 :
                cv2.rectangle(roi,((extTop[0]),extTop[1]),((extRight[0]),extRight[1]),(0,255,0),2)
            else:
                pass
        except:
            pass

    else:
        pass
    
    cv2.imshow("frame",frame)
    cv2.imshow("roi",roi)
    cv2.imshow("mask",mask)
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(1) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break
        
cap.release() 
cv2.destroyAllWindows()

# Göz bebiğini algılatmak

In [40]:
import cv2
import numpy as np

cap = cv2.VideoCapture("resimler/15.1 eye_motion.mp4.mp4")


while True:
    
    ret,frame = cap.read()
    if ret == 0:
        break
    roi = frame[80:210,230:450] # frame[y1:y2,x1:x2] ilgili alan roi
    rows,cols,_ =roi.shape
    cv2.rectangle(frame,(230,80),(450,210),(0,255,0),0)
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY) 
    kernel = np.ones((1,1),np.uint8) #birlerden oluşup beyazlar içindeki siyahları yok etmek için kullanılacak
    mask = cv2.dilate(gray,kernel,iterations =1)
    mask = cv2.medianBlur(mask,15)      
    _,thresh =cv2.threshold(mask,0,255,cv2.THRESH_BINARY_INV)
    contours,_ = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    contours =sorted(contours,key=lambda x:cv2.contourArea(x),reverse =True) #içindeki değerleri sıralıyor,Reverse True tersten sırala demek
    
    for cnt in contours:
        (x,y,w,h)=cv2.boundingRect(cnt) #bu komut ile kordinat değerlerini çekebiliyoruz
        
        cv2.rectangle(roi,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.line(roi,(x+int(w/2),0),(x+int(w/2),rows),(0,255,0),2)
        cv2.line(roi,(0,(y+int(h/2))),(cols,int(y+(h/2))),(0,255,0),2)
        break
    
    
    #frame[80:210,230:450] =roi
    cv2.imshow("frame",frame)
    cv2.imshow("thresh",thresh)
    cv2.imshow("roi",roi)
    
   
    
 
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(80) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break
    
cap.release() 
cv2.destroyAllWindows()

# Resim Karşılaştırmak

In [51]:
import cv2
import numpy as np

img = cv2.imread("resimler/1.1 aircraft.jpg.jpg")
img = cv2.resize(img,(640,500)) # (weight ,height)

img1 =cv2.imread("resimler/aircopy.jpg")
img1 = cv2.resize(img1,(640,500)) # (weight ,height)

if img1.shape == img.shape:
    print("same size")
    
    #diff =difference =farklılık
    
    diff = cv2.subtract(img,img1) # subtract farklılıkları bulur ve bize döndürür
    b,g,r = cv2.split(diff) #renk değerlerini parçalıyor
    
    if cv2.countNonZero(b)==0 and cv2.countNonZero(g)==0 and cv2.countNonZero(r)==0 :
        print("Completely equal")
    else:
        print("NOT Completely equal")
    
else:
    print("not same")
    



cv2.imshow("original",img)
cv2.imshow("kopya",img1)
cv2.imshow("fark",diff)
cv2.waitKey(0)
cv2.destroyAllWindows()

same size
NOT Completely equal


# Çözünürlük Değerleri Ayarlamak

In [3]:
import cv2



cap = cv2.VideoCapture(0)

print("Width: "+str(cap.get(3))) # cap.get(3) bize görünütünün genişliğini döndürür
print("Height: "+str(cap.get(4))) # cap.get(4) bize görünütünün uzunluğunu döndürür

cap.set(3,1280)
cap.set(4,720)

print("Width*: "+str(cap.get(3))) # cap.get(3) bize görünütünün genişliğini döndürür
print("Height*: "+str(cap.get(4))) # cap.get(4) bize görünütünün uzunluğunu döndürür
while True:
    
    ret,frame = cap.read()
    
    
    frame = cv2.flip(frame,1)
    
    
    
    
    cv2.imshow("Live Video",frame)
    
    
 
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(11) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break
    
cap.release() 
cv2.destroyAllWindows()

Width: 640.0
Height: 480.0
Width*: 1280.0
Height*: 720.0


# Blur yapılmış resmi tespit etmek

In [9]:
import cv2

img = cv2.imread("resimler/5.1 starwars.jpg.jpg")
blur = cv2.medianBlur(img,7)

laplacian = cv2.Laplacian(blur,cv2.CV_64F).var() # bize bir değer döndürür
print(laplacian)
#elde edilen değerleri bakarak şu işlem yapılabilir

if laplacian < 500:
    print("blurry image")
else :
    print("not detect blur")


cv2.imshow("img",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

317.9994595426443
blurry image


# Şablon eşletirmek

In [16]:
import cv2
import numpy as np

img = cv2.imread("resimler/5.1 starwars.jpg.jpg")

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

temp =cv2.imread("resimler/5.2 starwars2.jpg.jpg",0)
print(temp.shape)  # (height,weight)
print(temp.shape[::-1])  # (weight,height)
w,h = temp.shape[::-1]  # (weight,height)

result = cv2.matchTemplate(gray,temp,cv2.TM_CCOEFF_NORMED)
#print(result)

location=np.where(result>=0.95) # resimde yoğunluğu bire yakın olan yerleri bulmalıyız
for point in zip(*location[::-1]): #yükseklik,genişlik olarak değerlei aldık ve diziye dönüştürdük
    print(point)
    cv2.rectangle(img,point,(point[0]+w,point[1]+h),(0,255,0),3)

cv2.imshow("temp",temp)
cv2.imshow("img",img)
cv2.imshow("res",result)

cv2.waitKey(0)
cv2.destroyAllWindows()

(117, 121)
(121, 117)
(282, 53)


In [ ]:
import cv2

img = cv2.imread("resimler/memo.jpg")
gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
w,h = gray_img.shape[::-1] # (weight,height)
print( gray_img.shape)
cap = cv2.VideoCapture(0) # videocaptura sıfır diyerek kameradan görüntü alacağımızı belirtilir


# her bir kare sonsuz bir while döngüsü ile yakalayarak video oluşturuyoruz.
while True:
    ret,frame = cap.read() # cap.read() sağlıklı bi şekilde görüntü okuduğunda true değerleri göndererek ret hafızasında saklanır
    
    #flipcode = 0: flip vertically
    #flipcode > 0: flip horizontally
    #flipcode < 0: flip vertically and horizontally
    
    frame = cv2.flip(frame,1)  #cv2.flip kodu ile görüntünün eksenlere göre tersini alabiliriz
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    result = cv2.matchTemplate(gray,gray_img,cv2.TM_CCOEFF_NORMED)
    #print(result)

    location=np.where(result>=0.7) # resimde yoğunluğu bire yakın olan yerleri bulmalıyız
    for point in zip(*location[::-1]): #yükseklik,genişlik olarak değerlei aldık ve diziye dönüştürdük
        cv2.rectangle(frame,point,(point[0]+w,point[1]+h),(0,255,0),3)

    cv2.imshow("webcam",frame)
        
    
    #burada waitkey içine girlen sayı her bir karenin kaç milisaniye gösterileceğidir
    if cv2.waitKey(1) & 0xFF == ord("q"): #0xFF q'nun makine dilinde ki karşılığıdır
        break


cap.release() 
cv2.destroyAllWindows()

# Resim üzerine resim eklemek

In [ ]:
import cv2

def nothing(x):
    pass

img =cv2.imread("resimler/1.1 aircraft.jpg.jpg")
img =cv2.resize(img,(640,480))

img1 =cv2.imread("resimler/5.1 starwars.jpg.jpg")
img1 =cv2.resize(img1,(640,480))

output = cv2.addWeighted(img,0.5,img1,0.5,0) #iki resmi birbiri üzerine ekliyoruz ve karışım oranları veriyoruz

windowName ="Transition Program"
cv2.namedWindow(windowName)

cv2.createTrackbar("Alpha-Beta",windowName,0,1000,nothing)

while True:
    cv2.imshow(windowName,output)
    alpha = cv2.getTrackbarPos("Alpha-Beta",windowName)/1000 #alpha-beta 0-1 arasında değişir
    beta =1-alpha
    output = cv2.addWeighted(img,alpha,img1,beta,0) #iki resmi birbiri üzerine ekliyoruz ve karışım oranları veriyoruz
    print(alpha,beta)
    if cv2.waitKey(1)==27: #esc basıldığında pencereyi kapat
        break
cv2.destroyAllWindows( )